In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import getpass
from typing import Optional, Literal
import ccxt  # Still useful for live trading

# AI Integration Imports
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, MessagesState, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from huggingface_hub import InferenceClient

In [ ]:
# Configuration Setup
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

_set_if_undefined("BINANCE_API_KEY")  # For live trading
_set_if_undefined("BINANCE_API_SECRET")
_set_if_undefined("HF_API_KEY")

In [ ]:
# Initialize AI Client
class AIClient:
    def __init__(self):
        self.hf_client = InferenceClient(
            provider="nebius",
            api_key=os.environ["HF_API_KEY"],
        )
    
    async def get_ai_response(self, messages: list, model: str = "deepseek-ai/DeepSeek-V3-0324"):
        """Get AI response with error handling"""
        try:
            completion = self.hf_client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=512,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"AI Error: {str(e)}")
            return "hold|0"  # Default safe response

ai_client = AIClient()

In [ ]:
# SOL/USDT Data Loader
class SOLDataLoader:
    def __init__(self):
        # Load both datasets
        self.daily_data = pd.read_csv('SOLUSDT_day.csv', parse_dates=['date'])
        self.hourly_data = pd.read_csv('SOLUSDT_hour.csv', parse_dates=['date'])
        
        # Clean and prepare data
        self.daily_data.set_index('date', inplace=True)
        self.hourly_data.set_index('date', inplace=True)
        
        # Calculate technical indicators for both timeframes
        self._calculate_indicators(self.daily_data)
        self._calculate_indicators(self.hourly_data)
    
    def _calculate_indicators(self, data: pd.DataFrame):
        """Calculate technical indicators for SOL"""
        # Moving Averages
        data['sma_20'] = data['close'].rolling(20).mean()
        data['sma_50'] = data['close'].rolling(50).mean()
        data['sma_100'] = data['close'].rolling(100).mean()
        
        # RSI
        delta = data['close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(14).mean()
        avg_loss = loss.rolling(14).mean()
        rs = avg_gain / avg_loss
        data['rsi'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        data['upper_band'] = data['sma_20'] + (2 * data['close'].rolling(20).std())
        data['lower_band'] = data['sma_20'] - (2 * data['close'].rolling(20).std())
        
        # MACD
        exp12 = data['close'].ewm(span=12, adjust=False).mean()
        exp26 = data['close'].ewm(span=26, adjust=False).mean()
        data['macd'] = exp12 - exp26
        data['signal'] = data['macd'].ewm(span=9, adjust=False).mean()
        
        return data
    
    def get_data(self, start_date: str, end_date: str, timeframe: str = 'day') -> pd.DataFrame:
        """Get SOL data for specific timeframe and date range"""
        df = self.daily_data if timeframe == 'day' else self.hourly_data
        return df.loc[start_date:end_date].copy()

sol_data = SOLDataLoader()


In [ ]:
# Agent Nodes
async def market_data_agent(state: MessagesState):
    """Gathers and processes SOL market data"""
    messages = state["messages"]
    params = messages[-1].additional_kwargs
    
    # Get SOL data
    data = sol_data.get_data(
        params["start_date"],
        params["end_date"],
        params.get("timeframe", "day")
    )
    
    if len(data) < 1:
        raise ValueError("No SOL data available for this date range")
    
    last_row = data.iloc[-1]
    
    message = HumanMessage(
        content=f"""
        SOL/USDT Market Data ({params.get("timeframe", "day")} timeframe):
        Current Price: {last_row['close']:.4f}
        SMA 20: {last_row['sma_20']:.4f}
        SMA 50: {last_row['sma_50']:.4f}
        SMA 100: {last_row['sma_100']:.4f}
        RSI: {last_row['rsi']:.2f}
        Upper Band: {last_row['upper_band']:.4f}
        Lower Band: {last_row['lower_band']:.4f}
        MACD: {last_row['macd']:.4f}
        Signal: {last_row['signal']:.4f}
        Volume: {last_row['volume']:.2f}
        """,
        name="market_data_agent",
    )
    
    return {"messages": messages + [message]}

async def quant_agent(state: MessagesState):
    """Analyzes SOL technical indicators"""
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""You are a SOL/USDT trading expert analyzing market data.
        Provide analysis with:
        - signal: bullish | bearish | neutral
        - confidence: <float 0-1>
        - reasoning: <brief technical explanation>
        Consider SOL's volatility and correlation with Bitcoin."""),
        last_message
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": state["messages"] + [
        HumanMessage(content=f"SOL Analysis: {result}", name="quant_agent")
    ]}

async def risk_management_agent(state: MessagesState):
    """SOL-specific risk management"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""SOL Risk Manager:
        - Max position: 1-3% of capital (SOL is volatile)
        - Consider SOL's typical daily range (~5-15%)
        - Account for potential SOL network outages
        Output: max_position_size, risk_score(1-10)"""),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"""Analysis: {last_message.content}
        Portfolio: {portfolio['cash']:.2f} USDT | Position: {portfolio['position']:.2f} SOL""")
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": state["messages"] + [
        HumanMessage(content=f"Risk Assessment: {result}", name="risk_management")
    ]}

async def portfolio_management_agent(state: MessagesState):
    """Executes SOL trades with proper sizing"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""SOL Portfolio Manager:
        - Binance SOL/USDT minimum order: 0.1 SOL
        - Action: buy | sell | hold
        - Quantity: in SOL units
        - Maintain 1-2% risk per trade
        - Consider SOL's liquidity (usually good)"""),
        last_message,
        HumanMessage(content=f"""Risk Data: {last_message.content}
        Portfolio: {portfolio['cash']:.2f} USDT | Position: {portfolio['position']:.2f} SOL""")
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": [HumanMessage(content=result, name="portfolio_management")]}

In [ ]:
# Workflow Construction
workflow = StateGraph(MessagesState)
workflow.add_node("market_data_agent", market_data_agent)
workflow.add_node("quant_agent", quant_agent)
workflow.add_node("risk_management_agent", risk_management_agent)
workflow.add_node("portfolio_management_agent", portfolio_management_agent)

workflow.add_edge(START, "market_data_agent")
workflow.add_edge("market_data_agent", "quant_agent")
workflow.add_edge("quant_agent", "risk_management_agent")
workflow.add_edge("risk_management_agent", "portfolio_management_agent")
workflow.add_edge("portfolio_management_agent", END)

app = workflow.compile()


In [ ]:
# SOL Backtester
class SOLBacktester:
    def __init__(self, agent, start_date, end_date, initial_capital, timeframe='day'):
        self.agent = agent
        self.symbol = "SOL/USDT"
        self.start_date = start_date
        self.end_date = end_date
        self.timeframe = timeframe
        self.initial_capital = initial_capital
        self.portfolio = {"cash": initial_capital, "position": 0.0}
        self.portfolio_values = []
        self.trade_history = []
        self.last_valid_price = 20.0  # Reasonable SOL default

    def parse_action(self, output):
        try:
            import re
            action = "hold"
            quantity = 0.0

            action_match = re.search(r'action["\']?\s*:\s*["\']?(\w+)', output, re.IGNORECASE)
            quantity_match = re.search(r'quantity["\']?\s*:\s*([\d.]+)', output, re.IGNORECASE)

            if action_match:
                action = action_match.group(1).lower()
            if quantity_match:
                quantity = float(quantity_match.group(1))

            return action, max(0.01, quantity)  # Minimum 0.01 SOL
        except:
            return "hold", 0.0

    async def execute_trade(self, action, quantity, current_price):
        """Safe SOL trade execution with validation"""
        if current_price <= 0:
            return 0.0

        # Round to 2 decimals for SOL (Binance allows 0.01 precision)
        quantity = round(quantity, 2)

        if action == "buy":
            max_affordable = self.portfolio["cash"] / current_price
            quantity = min(quantity, max_affordable)
            if quantity >= 0.1:  # Binance minimum order size
                self.portfolio["position"] += quantity
                self.portfolio["cash"] -= quantity * current_price
                self.trade_history.append({
                    'date': datetime.now().strftime('%Y-%m-%d'),
                    'action': 'buy',
                    'quantity': quantity,
                    'price': current_price
                })
                return quantity
        elif action == "sell":
            quantity = min(quantity, self.portfolio["position"])
            if quantity >= 0.1:  # Binance minimum order size
                self.portfolio["position"] -= quantity
                self.portfolio["cash"] += quantity * current_price
                self.trade_history.append({
                    'date': datetime.now().strftime('%Y-%m-%d'),
                    'action': 'sell',
                    'quantity': quantity,
                    'price': current_price
                })
                return quantity
        return 0.0

    async def run_backtest(self):
        # Get all dates in range
        dates = sol_data.daily_data.loc[self.start_date:self.end_date].index
        
        print(f"\nStarting SOL/USDT Backtest ({self.timeframe} timeframe)...")
        print(f"{'Date':<12} {'Action':<6} {'Quantity':>10} {'Price':>10} {'Cash':>12} {'Position':>12} {'Value':>12}")
        print("-" * 78)

        for current_date in dates:
            current_date_str = current_date.strftime("%Y-%m-%d")
            lookback_start = (current_date - timedelta(days=30)).strftime("%Y-%m-%d")

            action = "hold"
            executed_qty = 0.0
            current_price = self.last_valid_price
            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            try:
                # Get current price
                current_data = sol_data.daily_data.loc[current_date_str]
                current_price = current_data['close']
                self.last_valid_price = current_price

                # Only trade on certain conditions (e.g., every 3 days for daily timeframe)
                if self.timeframe == 'day' and current_date.weekday() not in [0, 3]:  # Monday, Thursday
                    total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price
                    self.portfolio_values.append({
                        "Date": current_date,
                        "Portfolio Value": total_value
                    })
                    continue

                # Get agent decision
                agent_output = await self.agent({
                    "messages": [HumanMessage(
                        content="Make a trading decision",
                        additional_kwargs={
                            "symbol": self.symbol,
                            "start_date": lookback_start,
                            "end_date": current_date_str,
                            "timeframe": self.timeframe,
                            "portfolio": self.portfolio.copy()
                        }
                    )]
                })
                final_output = agent_output["messages"][-1].content

                # Parse and execute trade
                action, quantity = self.parse_action(final_output)
                executed_qty = await self.execute_trade(action, quantity, current_price)

            except Exception as e:
                print(f"Error processing {current_date_str}: {str(e)}")
                current_price = self.last_valid_price

            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            self.portfolio_values.append({
                "Date": current_date,
                "Portfolio Value": total_value
            })

            print(f"{current_date_str} "
                  f"{action:<6} {executed_qty:>10.2f} {current_price:>10.4f} "
                  f"{self.portfolio['cash']:>12.2f} {self.portfolio['position']:>12.2f} {total_value:>12.2f}")

    def analyze_performance(self):
        if not self.portfolio_values:
            print("No backtest results to analyze")
            return None

        performance_df = pd.DataFrame(self.portfolio_values)
        performance_df.set_index('Date', inplace=True)

        # Calculate metrics
        initial = self.initial_capital
        final = performance_df['Portfolio Value'].iloc[-1]
        total_return = (final - initial) / initial
        
        # Calculate Sharpe Ratio (assuming 0% risk-free rate)
        returns = performance_df['Portfolio Value'].pct_change().dropna()
        sharpe_ratio = returns.mean() / returns.std() * np.sqrt(365)
        
        # Max Drawdown
        peak = performance_df['Portfolio Value'].cummax()
        drawdown = (performance_df['Portfolio Value'] - peak) / peak
        max_drawdown = drawdown.min()

        print(f"\nSOL/USDT Backtest Results ({self.timeframe} timeframe)")
        print(f"Initial Capital: ${initial:,.2f}")
        print(f"Final Value: ${final:,.2f}")
        print(f"Total Return: {total_return*100:.2f}%")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        print(f"Max Drawdown: {max_drawdown*100:.2f}%")

        # Plot performance
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [2, 1]})
        
        # Portfolio Value
        performance_df["Portfolio Value"].plot(ax=ax1, title="SOL/USDT Portfolio Performance")
        ax1.set_ylabel("Value (USDT)")
        
        # Drawdown
        drawdown.plot(ax=ax2, title="Portfolio Drawdown", color='red')
        ax2.set_ylabel("Drawdown")
        ax2.set_ylim(-1, 0)
        
        plt.tight_layout()
        plt.show()

        return performance_df



In [ ]:
# Run the backtest
async def main():
    print("Available SOL/USDT data from 2020-09-14 to 2025-04-08")
    
    # Example backtest configuration
    backtester = SOLBacktester(
        agent=app,
        start_date="2024-01-01",
        end_date="2024-12-31",
        initial_capital=1000,
        timeframe='day'  # or 'hour'
    )
    
    await backtester.run_backtest()
    backtester.analyze_performance()

    # Show trade history
    if hasattr(backtester, 'trade_history') and backtester.trade_history:
        print("\nTrade History:")
        for trade in backtester.trade_history:
            print(f"{trade['date']} {trade['action']:>5} {trade['quantity']:>6.2f} SOL @ {trade['price']:.4f}")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())